In [ ]:
import squidpy as sd
import scanpy as sc
# import pandas as pd
import numpy as np
# import anndata as ad
# import spatialdata_io as sio
from matplotlib.pyplot import plt
from pyprojroot import here


This notebook walks you through standard QC of Xenium data. This notebook will not cover region selection. It is assumed you are using this to check an entire sample, which can later be analyzed with a narrower focus. 

Some of this code is adapted from [10x's analysis guide](https://www.10xgenomics.com/analysis-guides/xenium-downstream-analysis-in-python-tutorial)

# Load data

In [ ]:
sdata = sd.read_zarr("breast.zarr/")
sdata

The spatial object contains images and cell boundary definitions, which take up a lot of space. We mostly work with the single-cell like anndata object. 

In [ ]:
adata = sdata["table"]
# We store the raw counts in the layers in case we need it for future purposes
adata.layers["counts"] = adata.X.copy()

# Filtering cells

We want to remove cells with low transcript counts. Here we plot the distribution of transcript counts per cell in a histogram to determine the transcript count cutoff to remove cells with low transcript counts. The determination of lower cutoff can be subjective. There is no gold-standard rule. 

In [ ]:
fig = plt.hist(adata.obs["total_counts"], range=(0, 200), bins=100)
plt.axvline(x=20, color="r", linestyle="--")

plt.xlabel("Transcripts per cell")
plt.ylabel("Number of cells")

In [ ]:
## Apply a hard cutoff for transcripts per cell, or use a quantile-based approach
thres = np.quantile(adata.obs["total_counts"], 0.98)
sc.pp.filter_cells(adata, min_counts=20)
sc.pp.filter_cells(adata, max_counts=thres)

## TODO: implement a nMAD approach for outlier detection

# Feature selection

In [ ]:
# We also filter out genes that are rarely expressed
# Adjust this value based on the # of cells in your sample
sc.pp.filter_genes(adata, min_cells=100)

If you have a large panel (>2k genes), defining highly variable genes is needed for downstream dimensional reduction

In [ ]:
if adata.n_vars > 2000:
    sc.pp.highly_variable_genes(adata, n_top_genes=2000)

# Normalize, log-transform, and scale

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata.layers["lognorm"] = adata.X.copy()

In [ ]:
# Scaling of data, we don't center the data here as the data centering is done in PCA and this keeps the X sparse for memory efficiency.
sc.pp.scale(adata, zero_center=False, max_value=10)

In [ ]:
# PCA for dimension reduction further by UMAP for downstream analysis
sc.pp.pca(adata, n_comps=30)
sc.pp.neighbors(adata, metric="cosine")
# clustering with leiden. We use igraph here as it scales better. The resolution is a hyper-parameter. A higher resolution will output more clusters.
sc.tl.leiden(adata, flavor="igraph", n_iterations=-1, resolution=0.5)

# UMAP

In [ ]:
## this may take a while
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color="leiden")

In [ ]:
sdata.tables["table"].obs["region"] = "cell_boundaries"
sdata.set_table_annotates_spatialelement("table", region="cell_boundaries")

sdata.pl.render_shapes("cell_boundaries", color="leiden").pl.show(figsize=(12.8, 9.6))

# Save

In [ ]:
# sdata.write("breast_processed.zarr", overwrite=True)
## Alternatively, could just save the adata for lighter footprint
adata.write(here('03_analysis/04_checkpoints/01_qc-filtered.h5ad'))
